# TITLE OF NOTEBOOK

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import sklearn 

In [ ]:

dataset_23 = pd.read_csv("datasets/2022-2023_Football_Player_Stats.csv", encoding="latin1")
dataset_22 = pd.read_csv("datasets/2021-2021_Football_Player_Stats.csv", encoding="latin1")
